In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        img = np.dstack((band_1, band_2, band_3))
        images.append(std_img(img))

    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')


model model


In [5]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.70321, saving model to best_m.h5
 - 3s - loss: 0.6820 - acc: 0.5536 - val_loss: 0.7032 - val_acc: 0.4913
Epoch 2/100
Epoch 00002: val_loss improved from 0.70321 to 0.43996, saving model to best_m.h5
 - 2s - loss: 0.6015 - acc: 0.6474 - val_loss: 0.4400 - val_acc: 0.8155
Epoch 3/100
Epoch 00003: val_loss improved from 0.43996 to 0.42348, saving model to best_m.h5
 - 2s - loss: 0.5070 - acc: 0.7491 - val_loss: 0.4235 - val_acc: 0.8254
Epoch 4/100
Epoch 00004: val_loss improved from 0.42348 to 0.37630, saving model to best_m.h5
 - 2s - loss: 0.4860 - acc: 0.7572 - val_loss: 0.3763 - val_acc: 0.8454
Epoch 5/100
Epoch 00005: val_loss improved from 0.37630 to 0.32262, saving model to best_m.h5
 - 2s - loss: 0.5057 - acc: 0.7641 - val_loss: 0.3226 - val_acc: 0.8579
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 2s - loss: 0.4354 - acc: 0.8025 - val_loss: 0.3524 - val_acc: 0.8579
Epoch 7/100
Epoch 00007: val_loss did not improve
 -

Epoch 62/100
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.2633 - acc: 0.8914 - val_loss: 0.1922 - val_acc: 0.9352
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 2s - loss: 0.2384 - acc: 0.9033 - val_loss: 0.1972 - val_acc: 0.9152
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.2590 - acc: 0.8942 - val_loss: 0.1958 - val_acc: 0.9227
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.2473 - acc: 0.8964 - val_loss: 0.1859 - val_acc: 0.9202
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.2313 - acc: 0.8906 - val_loss: 0.1844 - val_acc: 0.9177
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.2505 - acc: 0.8914 - val_loss: 0.1868 - val_acc: 0.9177
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.2544 - acc: 0.8873 - val_loss: 0.1931 - val_acc: 0.9177
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.2369 - acc: 0.8983 - val_loss: 0.1869 - val_acc: 0.9252
Epoch 70/100
Epo

Epoch 23/100
Epoch 00023: val_loss improved from 0.27375 to 0.27151, saving model to best_m.h5
 - 2s - loss: 0.2706 - acc: 0.8766 - val_loss: 0.2715 - val_acc: 0.8678
Epoch 24/100
Epoch 00024: val_loss improved from 0.27151 to 0.26454, saving model to best_m.h5
 - 2s - loss: 0.2894 - acc: 0.8908 - val_loss: 0.2645 - val_acc: 0.8828
Epoch 25/100
Epoch 00025: val_loss did not improve
 - 2s - loss: 0.3064 - acc: 0.8758 - val_loss: 0.2851 - val_acc: 0.8579
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 2s - loss: 0.2659 - acc: 0.8756 - val_loss: 0.2810 - val_acc: 0.8803
Epoch 27/100
Epoch 00027: val_loss improved from 0.26454 to 0.25927, saving model to best_m.h5
 - 2s - loss: 0.2705 - acc: 0.8900 - val_loss: 0.2593 - val_acc: 0.8828
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 2s - loss: 0.2857 - acc: 0.8792 - val_loss: 0.2607 - val_acc: 0.8703
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 2s - loss: 0.2439 - acc: 0.8925 - val_loss: 0.2716 - val_acc: 0.8878
Epoch 3

Epoch 86/100
Epoch 00086: val_loss did not improve
 - 2s - loss: 0.1968 - acc: 0.9058 - val_loss: 0.2419 - val_acc: 0.8853
Epoch 87/100
Epoch 00087: val_loss did not improve
 - 2s - loss: 0.1980 - acc: 0.9067 - val_loss: 0.2764 - val_acc: 0.8628
Epoch 88/100
Epoch 00088: val_loss did not improve
 - 2s - loss: 0.2153 - acc: 0.9008 - val_loss: 0.2474 - val_acc: 0.8928
Epoch 89/100
Epoch 00089: val_loss did not improve
 - 2s - loss: 0.2223 - acc: 0.9058 - val_loss: 0.2362 - val_acc: 0.9002
Epoch 90/100
Epoch 00090: val_loss did not improve
 - 2s - loss: 0.2170 - acc: 0.9125 - val_loss: 0.2341 - val_acc: 0.8928
Epoch 91/100
Epoch 00091: val_loss did not improve
 - 2s - loss: 0.2121 - acc: 0.9108 - val_loss: 0.2369 - val_acc: 0.8953
Epoch 92/100
Epoch 00092: val_loss improved from 0.23342 to 0.23240, saving model to best_m.h5
 - 2s - loss: 0.2155 - acc: 0.9042 - val_loss: 0.2324 - val_acc: 0.8953
Epoch 93/100
Epoch 00093: val_loss did not improve
 - 2s - loss: 0.2203 - acc: 0.9033 - val_los

Epoch 46/100
Epoch 00046: val_loss did not improve
 - 2s - loss: 0.2517 - acc: 0.8958 - val_loss: 0.2193 - val_acc: 0.9052
Epoch 47/100
Epoch 00047: val_loss improved from 0.21827 to 0.21626, saving model to best_m.h5
 - 2s - loss: 0.2258 - acc: 0.9042 - val_loss: 0.2163 - val_acc: 0.9077
Epoch 48/100
Epoch 00048: val_loss did not improve
 - 2s - loss: 0.2696 - acc: 0.8933 - val_loss: 0.2330 - val_acc: 0.8978
Epoch 49/100
Epoch 00049: val_loss did not improve
 - 2s - loss: 0.2523 - acc: 0.8883 - val_loss: 0.2286 - val_acc: 0.8978
Epoch 50/100
Epoch 00050: val_loss did not improve
 - 2s - loss: 0.2571 - acc: 0.8914 - val_loss: 0.2480 - val_acc: 0.8953
Epoch 51/100
Epoch 00051: val_loss did not improve
 - 2s - loss: 0.2525 - acc: 0.8939 - val_loss: 0.2334 - val_acc: 0.8953
Epoch 52/100
Epoch 00052: val_loss did not improve
 - 2s - loss: 0.2328 - acc: 0.8998 - val_loss: 0.2196 - val_acc: 0.9002
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 2s - loss: 0.2539 - acc: 0.8883 - val_los

Epoch 10/100
Epoch 00010: val_loss improved from 0.34305 to 0.33122, saving model to best_m.h5
 - 2s - loss: 0.3472 - acc: 0.8275 - val_loss: 0.3312 - val_acc: 0.8429
Epoch 11/100
Epoch 00011: val_loss improved from 0.33122 to 0.32128, saving model to best_m.h5
 - 2s - loss: 0.3202 - acc: 0.8516 - val_loss: 0.3213 - val_acc: 0.8753
Epoch 12/100
Epoch 00012: val_loss did not improve
 - 2s - loss: 0.3020 - acc: 0.8650 - val_loss: 0.3393 - val_acc: 0.8828
Epoch 13/100
Epoch 00013: val_loss did not improve
 - 2s - loss: 0.2962 - acc: 0.8716 - val_loss: 0.3477 - val_acc: 0.8828
Epoch 14/100
Epoch 00014: val_loss improved from 0.32128 to 0.31907, saving model to best_m.h5
 - 2s - loss: 0.3125 - acc: 0.8462 - val_loss: 0.3191 - val_acc: 0.8703
Epoch 15/100
Epoch 00015: val_loss improved from 0.31907 to 0.31004, saving model to best_m.h5
 - 2s - loss: 0.2824 - acc: 0.8750 - val_loss: 0.3100 - val_acc: 0.8703
Epoch 16/100
Epoch 00016: val_loss did not improve
 - 2s - loss: 0.2907 - acc: 0.8791 

Epoch 71/100
Epoch 00071: val_loss did not improve
 - 2s - loss: 0.2170 - acc: 0.9075 - val_loss: 0.2477 - val_acc: 0.9227
Epoch 72/100
Epoch 00072: val_loss did not improve
 - 2s - loss: 0.2223 - acc: 0.9008 - val_loss: 0.2429 - val_acc: 0.9177
Epoch 73/100
Epoch 00073: val_loss did not improve
 - 2s - loss: 0.1954 - acc: 0.9100 - val_loss: 0.2470 - val_acc: 0.9152
Epoch 74/100
Epoch 00074: val_loss did not improve
 - 2s - loss: 0.2009 - acc: 0.9050 - val_loss: 0.2451 - val_acc: 0.9252
Epoch 75/100
Epoch 00075: val_loss did not improve
 - 2s - loss: 0.1888 - acc: 0.9158 - val_loss: 0.2726 - val_acc: 0.9002
Epoch 76/100
Epoch 00076: val_loss did not improve
 - 2s - loss: 0.2091 - acc: 0.9067 - val_loss: 0.2481 - val_acc: 0.9227
Epoch 77/100
Epoch 00077: val_loss did not improve
 - 2s - loss: 0.2036 - acc: 0.9108 - val_loss: 0.2454 - val_acc: 0.9302
Epoch 78/100
Epoch 00078: val_loss did not improve
 - 2s - loss: 0.1848 - acc: 0.9283 - val_loss: 0.2498 - val_acc: 0.9202
Epoch 79/100
Epo

In [6]:
import pickle
with open('../features/cnn_3_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_3_aug1_sub.csv', index=False)
# pre 228
# new 2117

0.211734517896
         id  is_iceberg
0  5941774d    0.121678
1  4023181e    0.582949
2  b20200e4    0.083990
3  e7f018bb    0.997006
4  4371c8c3    0.059046
